<a href="https://colab.research.google.com/github/i-ganza007/PacMan_Formative/blob/main/CNNPOLICY_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update && apt-get install swig cmake
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install box2d-py tensorflow keras keras-rl2 gym

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,125 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,441 kB]  
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-dri

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

import ale_py
import gymnasium as gym
gym.register_envs(ale_py)
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
import numpy as np
import random
import torch
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
CONFIG_ID = 1

configs = [
    {"lr": 0.0001, "gamma": 0.99,  "batch": 8,  "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.3},
    {"lr": 0.0005, "gamma": 0.95,  "batch": 16, "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.2},
    {"lr": 0.0002, "gamma": 0.99,  "batch": 8,  "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.4},
    {"lr": 0.0003, "gamma": 0.999, "batch": 12, "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.25},
    {"lr": 0.00015,"gamma": 0.98,  "batch": 8,  "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.35},
    {"lr": 0.0004, "gamma": 0.99,  "batch": 16, "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.3},
    {"lr": 0.0001, "gamma": 0.995, "batch": 12, "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.2},
    {"lr": 0.0002, "gamma": 0.98,  "batch": 8,  "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.5},
    {"lr": 0.00025,"gamma": 0.99,  "batch": 16, "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.3},
    {"lr": 0.0003, "gamma": 0.999, "batch": 12, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.25},
]

cfg = configs[CONFIG_ID - 1]


print(f"Policy: CnnPolicy")
print(f"Hyperparameters:")
print(f"  • Learning Rate (lr): {cfg['lr']}")
print(f"  • Gamma (γ): {cfg['gamma']}")
print(f"  • Batch Size: {cfg['batch']}")
print(f"  • Epsilon Start: {cfg['eps_start']}")
print(f"  • Epsilon End: {cfg['eps_end']}")
print(f"  • Epsilon Decay: {cfg['eps_decay']}")


Policy: CnnPolicy
Hyperparameters:
  • Learning Rate (lr): 0.0001
  • Gamma (γ): 0.99
  • Batch Size: 8
  • Epsilon Start: 1.0
  • Epsilon End: 0.05
  • Epsilon Decay: 0.3


In [ ]:
env_test = gym.make("ALE/SpaceInvaders-v5")
random_scores = []

for episode in range(1, 21):
    obs, _ = env_test.reset()
    done = False
    truncated = False
    score = 0
    while not (done or truncated):
        action = random.choice([0, 1, 2, 3, 4, 5])
        obs, reward, done, truncated, _ = env_test.step(action)
        score += reward
    random_scores.append(score)
    if episode % 5 == 0:
        print(f"  Episode {episode}: {score}")

avg_random_score = np.mean(random_scores)
print(f"\nRandom Agent Average Score: {avg_random_score:.2f}")
env_test.close()

  Episode 5: 20.0
  Episode 10: 80.0
  Episode 15: 160.0
  Episode 20: 90.0

Random Agent Average Score: 101.50


In [ ]:
env = gym.make("ALE/SpaceInvaders-v5", frameskip=4)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)


In [ ]:
model = DQN(
    "CnnPolicy",
    env,
    learning_rate=cfg["lr"],
    gamma=cfg["gamma"],
    batch_size=cfg["batch"],
    buffer_size=10000,
    learning_starts=5000,
    exploration_initial_eps=cfg["eps_start"],
    exploration_final_eps=cfg["eps_end"],
    exploration_fraction=cfg["eps_decay"],
    target_update_interval=1000,
    train_freq=4,
    gradient_steps=1,
    tensorboard_log=f"./logs/",
    verbose=1,
    device="cuda"
)

print("Model created successfully!")
print(f"   Buffer size: {model.buffer_size:,}")
print(f"   Learning starts at: {model.learning_starts:,} steps")
print(f"   Policy: CnnPolicy (Convolutional Neural Network)")

print(f"\nInitial GPU memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

Using cuda device
Wrapping the env in a VecTransposeImage.
Model created successfully!
   Buffer size: 10,000
   Learning starts at: 5,000 steps
   Policy: CnnPolicy (Convolutional Neural Network)

Initial GPU memory: 0.11 GB


In [ ]:
try:
    model.learn(
        total_timesteps=500_000,
        tb_log_name=f"SpaceInvaders_Config{CONFIG_ID}_CnnPolicy",
        log_interval=1000,
        progress_bar=True
    )

    print("\nTraining completed successfully!")
    print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print(f"\nOUT OF MEMORY ERROR")
        print(f"Current settings: buffer={model.buffer_size}, batch={cfg['batch']}")
        print(f"\nEMERGENCY FIX:")
        print(f"   1. Change buffer_size=5000")
        print(f"   2. Change batch_size=4")
        print(f"   3. Or use device='cpu' (slower but stable)")
    raise e

Logging to ./logs/SpaceInvaders_Config1_CnnPolicy_1


Output()


Training completed successfully!
Max GPU memory used: 0.29 GB


In [ ]:
mean_reward, std_reward = evaluate_policy(
    model,
    env,
    n_eval_episodes=10,
    deterministic=True
)

print(f"\nResults over 10 episodes:")
print(f"   Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"   Random Baseline: {avg_random_score:.2f}")
print(f"   Improvement: {mean_reward - avg_random_score:.2f}")


Results over 10 episodes:
   Mean Reward: 275.00 ± 182.28
   Random Baseline: 101.50
   Improvement: 173.50


In [ ]:
model_name = "dqn_model"
model.save(model_name)

backup_name = f"dqn_spaceinvaders_exp{CONFIG_ID}"
model.save(backup_name)

print(f"\nModels saved:")
print(f"   • {model_name}.zip (required submission)")
print(f"   • {backup_name}.zip (backup with config ID)")


Models saved:
   • dqn_model.zip (required submission)
   • dqn_spaceinvaders_exp1.zip (backup with config ID)
